## **ПРАКТИЧЕСКОЕ ЗАДАНИЕ**

**Задание 1.** 

Написать теггер на данных с русским языком

- проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
- написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
- сравнить все реализованные методы, сделать выводы
 
**Задание 2.** 

Проверить, насколько хорошо работает NER

Данные брать из http://www.labinform.ru/pub/named_entities/

- проверить NER из nltk/spacy/deeppavlov.
- написать свой NER, попробовать разные подходы.
 - передаём в сетку токен и его соседей.
 - передаём в сетку только токен.
 - свой вариант.
- сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.


### **ЗАДАНИЕ 1. Теггер на данных с русским языком.**

In [ ]:
# Загружаем библиотеки

import os
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import nltk
nltk.download('punkt')
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger_ru')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk.corpus import brown
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger, RegexpTagger
import warnings
warnings.filterwarnings("ignore")
import string
from nltk.tag.sequential import SequentialBackoffTagger
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import hstack, vstack

In [ ]:
# Вспомогательные функции

# Функция парсинга страницы
def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(['script', 'style', 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))


# Функция комбинирования теггеров
def backoff_tagger(train_sents, tagger_classes, backoff=None, patterns=None):
    for cls in tagger_classes:
      if cls == RegexpTagger:
        backoff = cls(patterns, backoff=backoff)
      else:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


# Кастомный теггер, размечающий английские слова в русском тексте как NONLEX
class EngTagger(SequentialBackoffTagger):

    def __init__(self, eng_set, backoff=None):
        self.eng_set = set([item.lower() for item in string.ascii_lowercase]) 
        super().__init__(backoff)

    def choose_tag(self, tokens, index, history):
        word = tokens[index]
        if word[0].lower() in self.eng_set:
             return 'NONLEX'
        else:
             return None

In [ ]:
compare_table = pd.DataFrame(columns=['Tagger', 'Score'])

In [ ]:
# Загрузим тренировочный и тестовый тексты

train = url_to_string('https://berza.ru/a-modular-spring-loaded-actuator/')
test = url_to_string('https://berza.ru/aws-panorama/')
train

' Создан ДНК-наноробот, который поможет изучать процессы в клетках                            Перейти к содержимому                          05.09.2022                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [ ]:
# Токенизируем тексты

train_tokens = nltk.word_tokenize(train, language='russian')
test_tokens = nltk.word_tokenize(test, language='russian')
train_tokens[:15]

['Создан',
 'ДНК-наноробот',
 ',',
 'который',
 'поможет',
 'изучать',
 'процессы',
 'в',
 'клетках',
 'Перейти',
 'к',
 'содержимому',
 '05.09.2022',
 'Берза',
 'Искусственный']

In [ ]:
# Сделаем теггирование текстов

train_data = [nltk.pos_tag(train_tokens, lang ='rus')]
test_data = [nltk.pos_tag(test_tokens, lang ='rus')]

In [ ]:
# Выделим теги

tags = [item[1] for item in train_data[0]]
tags[:15]

['V',
 'S',
 'NONLEX',
 'A-PRO=m',
 'V',
 'V',
 'S',
 'PR',
 'S',
 'V',
 'PR',
 'S',
 'NUM=ciph',
 'S',
 'A=m']

In [ ]:
# Посмотрим, как работает UnigramTagger

unigram_tagger = UnigramTagger(train_data)
display(unigram_tagger.tag(test_tokens)[:20], unigram_tagger.evaluate(test_data))

[('Создан', 'V'),
 ('искусственный', 'A=m'),
 ('интеллект', 'S'),
 (',', 'NONLEX'),
 ('который', 'A-PRO=m'),
 ('будет', None),
 ('контролировать', None),
 ('работников', None),
 ('Перейти', 'V'),
 ('к', 'PR'),
 ('содержимому', 'S'),
 ('05.09.2022', 'NUM=ciph'),
 ('Берза', 'S'),
 ('Искусственный', 'A=m'),
 ('интеллект', 'S'),
 ('и', 'CONJ'),
 ('машинное', 'A=n'),
 ('обучение', 'S'),
 (',', 'NONLEX'),
 ('чат-боты', 'S')]

0.5222672064777328

In [ ]:
compare_table.loc[0] = ['UnigramTagger', unigram_tagger.evaluate(test_data)]
compare_table

,Tagger,Score
0,UnigramTagger,0.522267


In [ ]:
# Посмотрим, как работает BigramTagger

bigram_tagger = BigramTagger(train_data, backoff=unigram_tagger)
display(bigram_tagger.tag(test_tokens)[:20], bigram_tagger.evaluate(test_data))

[('Создан', 'V'),
 ('искусственный', 'A=m'),
 ('интеллект', 'S'),
 (',', 'NONLEX'),
 ('который', 'A-PRO=m'),
 ('будет', None),
 ('контролировать', None),
 ('работников', None),
 ('Перейти', 'V'),
 ('к', 'PR'),
 ('содержимому', 'S'),
 ('05.09.2022', 'NUM=ciph'),
 ('Берза', 'S'),
 ('Искусственный', 'A=m'),
 ('интеллект', 'S'),
 ('и', 'CONJ'),
 ('машинное', 'A=n'),
 ('обучение', 'S'),
 (',', 'NONLEX'),
 ('чат-боты', 'S')]

0.5242914979757085

In [ ]:
compare_table.loc[1] = ['BigramTagger', bigram_tagger.evaluate(test_data)]

In [ ]:
# Посмотрим, как работает TrigramTagger

trigram_tagger = TrigramTagger(train_data, backoff=bigram_tagger)
display(trigram_tagger.tag(test_tokens)[:20], trigram_tagger.evaluate(test_data))

[('Создан', 'V'),
 ('искусственный', 'A=m'),
 ('интеллект', 'S'),
 (',', 'NONLEX'),
 ('который', 'A-PRO=m'),
 ('будет', None),
 ('контролировать', None),
 ('работников', None),
 ('Перейти', 'V'),
 ('к', 'PR'),
 ('содержимому', 'S'),
 ('05.09.2022', 'NUM=ciph'),
 ('Берза', 'S'),
 ('Искусственный', 'A=m'),
 ('интеллект', 'S'),
 ('и', 'CONJ'),
 ('машинное', 'A=n'),
 ('обучение', 'S'),
 (',', 'NONLEX'),
 ('чат-боты', 'S')]

0.5242914979757085

In [ ]:
compare_table.loc[2] = ['TrigramTagger', trigram_tagger.evaluate(test_data)]

In [ ]:
# Посмотрим, как работает вместе 3 варинта теггеров

backoff = DefaultTagger('NN')
combo_tagger = backoff_tagger(train_data,  
                              [UnigramTagger, BigramTagger, TrigramTagger],  
                              backoff=backoff)
display(combo_tagger.tag(test_tokens)[:20], combo_tagger.evaluate(test_data))

[('Создан', 'V'),
 ('искусственный', 'A=m'),
 ('интеллект', 'S'),
 (',', 'NONLEX'),
 ('который', 'A-PRO=m'),
 ('будет', 'NN'),
 ('контролировать', 'NN'),
 ('работников', 'NN'),
 ('Перейти', 'V'),
 ('к', 'PR'),
 ('содержимому', 'S'),
 ('05.09.2022', 'NUM=ciph'),
 ('Берза', 'S'),
 ('Искусственный', 'A=m'),
 ('интеллект', 'S'),
 ('и', 'CONJ'),
 ('машинное', 'A=n'),
 ('обучение', 'S'),
 (',', 'NONLEX'),
 ('чат-боты', 'S')]

0.5242914979757085

In [ ]:
compare_table.loc[3] = ['UBTTagger', combo_tagger.evaluate(test_data)]

In [ ]:
# Добавим кастомный теггер по разметке английских слов как NONLEX

backoff = DefaultTagger('NN')
combo2_tagger = backoff_tagger(train_data,  
                              [UnigramTagger, BigramTagger, TrigramTagger, EngTagger],  
                              backoff=backoff)
display(combo2_tagger.tag(test_tokens)[:20], combo2_tagger.evaluate(test_data))

[('Создан', 'V'),
 ('искусственный', 'A=m'),
 ('интеллект', 'S'),
 (',', 'NONLEX'),
 ('который', 'A-PRO=m'),
 ('будет', 'NN'),
 ('контролировать', 'NN'),
 ('работников', 'NN'),
 ('Перейти', 'V'),
 ('к', 'PR'),
 ('содержимому', 'S'),
 ('05.09.2022', 'NUM=ciph'),
 ('Берза', 'S'),
 ('Искусственный', 'A=m'),
 ('интеллект', 'S'),
 ('и', 'CONJ'),
 ('машинное', 'A=n'),
 ('обучение', 'S'),
 (',', 'NONLEX'),
 ('чат-боты', 'S')]

0.582995951417004

In [ ]:
compare_table.loc[4] = ['UBTETagger', combo2_tagger.evaluate(test_data)]

In [ ]:
# Добавим теггер RegexpTagger снекоторыми масками слов

patterns = [
    (r'.*[аи]ть$', 'V'),                
    (r'.*[ёеи]т$', 'V'),               
    (r'.*[и][ея]$', 'S'),                 
    (r'.*л[ю]$', 'S'),                
    (r'.*[ёе]нны*', 'A=m'),              
    (r'.*ческ*', 'A=m'),               
]

backoff = DefaultTagger('NN')
combo3_tagger = backoff_tagger(train_data,  
                              [RegexpTagger, UnigramTagger, BigramTagger, TrigramTagger, EngTagger],  
                              backoff=backoff, 
                              patterns=patterns)

display(combo3_tagger.tag(test_tokens)[:20], combo3_tagger.evaluate(test_data))

[('Создан', 'V'),
 ('искусственный', 'A=m'),
 ('интеллект', 'S'),
 (',', 'NONLEX'),
 ('который', 'A-PRO=m'),
 ('будет', 'V'),
 ('контролировать', 'V'),
 ('работников', 'NN'),
 ('Перейти', 'V'),
 ('к', 'PR'),
 ('содержимому', 'S'),
 ('05.09.2022', 'NUM=ciph'),
 ('Берза', 'S'),
 ('Искусственный', 'A=m'),
 ('интеллект', 'S'),
 ('и', 'CONJ'),
 ('машинное', 'A=n'),
 ('обучение', 'S'),
 (',', 'NONLEX'),
 ('чат-боты', 'S')]

0.6558704453441295

In [ ]:
compare_table.loc[5] = ['RUBTETagger', combo3_tagger.evaluate(test_data)]

In [ ]:
# Разделим токены и теги

train_tok = []
train_label = [] + ['NUM=acc', 'ADV-PRO', 'ANUM=m']
for sent in train_data[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in test_data[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [ ]:
# Закодируем теги в числа

le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)

In [ ]:
# Посмотрим на список тегов

le.classes_

array(['A-PRO', 'A-PRO=f', 'A-PRO=m', 'A-PRO=n', 'A-PRO=pl', 'A=f', 'A=m',
       'A=n', 'A=pl', 'ADV', 'ADV-PRO', 'ANUM=f', 'ANUM=m', 'CONJ',
       'NONLEX', 'NUM=acc', 'NUM=ciph', 'NUM=comp', 'NUM=gen', 'NUM=ins',
       'NUM=m', 'NUM=n', 'NUM=nom', 'PART', 'PR', 'S', 'S-PRO', 'V'],
      dtype='<U8')

In [ ]:
# Предскажем теги с помощью HashingVectorizer

hvectorizer = HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=250)
X_train_hash = hvectorizer.fit_transform(train_tok)
X_test_hash = hvectorizer.transform(test_tok)
lr_hash = LogisticRegression(random_state=0, max_iter=10)
lr_hash.fit(X_train_hash, train_enc_labels[:X_train_hash.shape[0]])
pred_hash = lr_hash.predict(X_test_hash)
accuracy_score(pred_hash, test_enc_labels)

0.4089068825910931

In [ ]:
compare_table.loc[6] = ['HVchar', accuracy_score(pred_hash, test_enc_labels)]

In [ ]:
# Предскажем теги с помощью HashingVectorizer

hvectorizer = HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=250)
X_train_hash = hvectorizer.fit_transform(train_tok)
X_test_hash = hvectorizer.transform(test_tok)
lr_hash = LogisticRegression(random_state=0, max_iter=10)
lr_hash.fit(X_train_hash, train_enc_labels[:X_train_hash.shape[0]])
pred_hash = lr_hash.predict(X_test_hash)
accuracy_score(pred_hash, test_enc_labels)

0.34210526315789475

In [ ]:
compare_table.loc[7] = ['HVword', accuracy_score(pred_hash, test_enc_labels)]

In [ ]:
# Предскажем теги с помощью TfidfVectorizer

vectorizer_char = TfidfVectorizer(ngram_range=(1, 5), analyzer='char')
X_train_tfidf_char = vectorizer_char.fit_transform(train_tok)
X_test_tfidf_char = vectorizer_char.transform(test_tok)
lr_tfidf = LogisticRegression(random_state=0, max_iter=10)
lr_tfidf.fit(X_train_tfidf_char, train_enc_labels[:X_train_hash.shape[0]])
pred_tfidf = lr_tfidf.predict(X_test_tfidf_char)
accuracy_score(pred_tfidf, test_enc_labels)

0.38866396761133604

In [ ]:
compare_table.loc[8] = ['TVchar', accuracy_score(pred_tfidf, test_enc_labels)]

In [ ]:
# Предскажем теги с помощью TfidfVectorizer

vectorizer_word = TfidfVectorizer(ngram_range=(1, 5), analyzer='word')
X_train_tfidf_word = vectorizer_word.fit_transform(train_tok)
X_test_tfidf_word = vectorizer_word.transform(test_tok)
lr_tfidf = LogisticRegression(random_state=0, max_iter=10)
lr_tfidf.fit(X_train_tfidf_word, train_enc_labels[:X_train_hash.shape[0]])
pred_tfidf = lr_tfidf.predict(X_test_tfidf_word)
accuracy_score(pred_tfidf, test_enc_labels)

0.37044534412955465

In [ ]:
compare_table.loc[9] = ['TVword', accuracy_score(pred_tfidf, test_enc_labels)]

In [ ]:
# Предскажем теги с помощью TfidfVectorizer, соединим фичи по символам и по словам

X_train_common = hstack((X_train_tfidf_char, X_train_tfidf_word))
X_test_common = hstack((X_test_tfidf_char, X_test_tfidf_word))
lr_tfidf = LogisticRegression(random_state=0, max_iter=10)
lr_tfidf.fit(X_train_common, train_enc_labels[:X_train_common.shape[0]])
pred_tfidf = lr_tfidf.predict(X_test_common)
accuracy_score(pred_tfidf, test_enc_labels)

0.3562753036437247

In [ ]:
compare_table.loc[10] = ['TVchar_word', accuracy_score(pred_tfidf, test_enc_labels)]

In [ ]:
compare_table

,Tagger,Score
0,UnigramTagger,0.522267
1,BigramTagger,0.524291
2,TrigramTagger,0.524291
3,UBTTagger,0.524291
4,UBTETagger,0.582996
5,RUBTETagger,0.655870
6,HVchar,0.408907
7,HVword,0.342105
8,TVchar,0.388664
9,TVword,0.370445


### **Вывод:**

Точность разбиения с помощью UnigramTagger равна 0.52 относительно nltk.pos_tag для русского языка. Метод BigramTagger помог улучшить точность, однако метод TrigramTagger и комбинирование всех трех методов не дали какого-либо результата. Обработка дополнительно английских слов с помощью кастомного теггера и применение метода регулярных выражений существенно улучшили качество разметки. Удалось поднять точность до 0.65.

Предсказания с помощью векторайзеров и логистической регрессии получились очень плохими.

**Что можно улучшить:**

* Попробовать взять другой текст с разметкой, в nltk.pos_tag используются довольно сложные метки, чтобы на них ориентироваться для подсчета точности собственного теггера (например, 'A-PRO=m', 'A=m').
* Добавить больше масок для метода RegexpTagger, чтобы обработать больше разных слов.

### **ЗАДАНИЕ 2. Проверить, насколько хорошо работает NER.**


**Задание 2.** 

Проверить, насколько хорошо работает NER

Данные брать из http://www.labinform.ru/pub/named_entities/

- проверить NER из nltk/spacy/deeppavlov.
- написать свой NER, попробовать разные подходы.
 - передаём в сетку токен и его соседей.
 - передаём в сетку только токен.
 - свой вариант.
- сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.


In [ ]:
!wget http://www.labinform.ru/pub/named_entities/collection3.zip
!unzip /content/collection3.zip

In [ ]:
DATA_ROOT = '/content/Collection3'

data_text = []
data_ann = []
for root, dirs, files in os.walk(DATA_ROOT):     
    for name in files:
        file_path_txt = os.path.join(root, name)
        file_path_ann = os.path.join(root, name[:-4]+'.ann')
        if 'txt'in name:
          with open(file_path_txt) as f:
            data_text.append(f.read())
          with open(file_path_ann) as f2:
            data_ann.append(f2.read())

print(f'Txt_length: {len(data_text)}')
print(f'Ann_length: {len(data_ann)}')

Txt_length: 1000
Ann_length: 1000


In [ ]:
orig_ner = list({(word, tag.split(' ')[0]) for (_, tag, word) in [item.split('\t') for item in data_ann[0].split('\n')][:-1]})
orig_ner

[('ФСФР', 'ORG'),
 ('Бурыкина', 'PER'),
 ('ЦБ', 'ORG'),
 ('Константина Шора', 'PER'),
 ('Алексей Моисеев', 'PER'),
 ('Сергей Швецов', 'PER'),
 ('Банка России', 'ORG'),
 ('Шор', 'PER'),
 ('РФ', 'LOC'),
 ('Наталья Бурыкина', 'PER'),
 ('Алексея Улюкаева', 'PER'),
 ('Улюкаев', 'PER'),
 ('Швецов', 'PER'),
 ('Госдумы', 'ORG')]

In [ ]:
# nltk ner

nltk_pos_tag = nltk.pos_tag(nltk.word_tokenize(data_text[0]))
nltk_ner = list({(' '.join(c[0] for c in chunk), chunk.label()) for chunk in nltk.ne_chunk(nltk_pos_tag) if hasattr(chunk, 'label')})
nltk_ner

[('ЦБ', 'ORGANIZATION'),
 ('Швецов', 'PERSON'),
 ('Совдир', 'PERSON'),
 ('Алексей Моисеев', 'PERSON'),
 ('Наталья Бурыкина', 'PERSON'),
 ('Улюкаев', 'PERSON'),
 ('Швецов', 'GPE'),
 ('Алексея Улюкаева', 'PERSON'),
 ('России', 'PERSON'),
 ('Госдумы', 'PERSON'),
 ('Сергей Швецов', 'PERSON')]

In [ ]:
# spacy ner

# !pip install -U spacy
# !python -m spacy info
# !python -m spacy download en_core_web_md
# !pip install en_core_web_md
import spacy
from spacy import displacy
import en_core_web_md

In [ ]:
nlp = en_core_web_md.load()
# ny_bb = url_to_string('https://www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news')
ny_bb = data_text[0]
article = nlp(ny_bb)
displacy.render(article, jupyter=True, style='ent')